<a href="https://colab.research.google.com/github/deethereal/NN/blob/main/%20mownfw/my_fw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import numpy as np

class Tensor(object):
  def __init__(self,  data, autograd=False,creators = None, creation_op = None, id =None):
    self.data = np.array(data)
    self.creation_op=creation_op  #как был создан этот тензор
    self.creators=creators #родители этого тензора
    self.grad=None
    self.autograd=autograd
    self.children={}
    if (id is None):
      id=np.random.randint(0,100000000)
    self.id=id
    if creators is not None:
      for c in creators:
        if (self.id not in c.children):
          c.children[self.id]=1
        else:
          c.children[self.id]+=1
  def all_children_grads_accounted_for(self):
    for id, cnt in self.children.items():
      if (cnt!=0):
        return False
    return True
  def backward(self,grad, grad_origin=None):
    if (self.autograd):
      if (grad_origin is not None):
        if (self.children[grad_origin.id] ==0):
          raise Exception ('cannot backpop more once')
        else:
          self.children[grad_origin.id]-=1
      if self.grad is None:
        self.grad=grad
      else:
        self.grad+=grad
      if (self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
        if self.creation_op == "add":
          self.creators[0].backward(grad) 
          self.creators[1].backward(grad)
        if self.creation_op == "sub":
          new =Tensor(self.grad.data)
          self.creators[0].backward(new,self)
          new =  Tensor(self.grad.__neg__().data)
          self.creators[1].backward(new,self)
        if self.creation_op == "neg":
          self.creators[0].backward(self.grad.__neg__()) 
        if self.creation_op == "mul":
          new=self.grad*self.creators[1]
          self.creators[0].backward(new,self) 
          new=self.grad*self.creators[0]
          self.creators[1].backward(new,self)
        if self.creation_op == "mm":
          act=self.creators[0]
          weights=self.creators[1]
          new = self.grad.mm(weights.transpose())
          act.backward(new)
          new=self.grad.transpose().mm(act).transpose()
          weights.backward(new)
        if self.creation_op == "transpose":
          self.creators[0].backward(self.grad.transpose())
        if ('sum' in self.creation_op):
          dim=int(self.creation_op.split('_')[1])
          ds=self.creators[0].data.shape[dim]
          self.creators[0].backward(self.grad.expand(dim,ds))
        if ('expand' in self.creation_op):
          dim=int(self.creation_op.split('_')[1])
          self.creators[0].backward(self.grad.sum(dim))

  def __add__(self,other):
    if self.autograd and other.autograd:
      return Tensor(self.data+other.data, autograd=True, creators=[self,other],creation_op='add')
    return Tensor(self.data+other.data)
  def __neg__(self):
    if self.autograd:
      return Tensor(self.data*-1, autograd=True, creators=[self],creation_op='neg')
    return Tensor(self.data*-1)
  def __sub__(self,other):
    if(self.autograd and other.autograd):
      return Tensor(self.data-other.data,autograd=True,creators=[self,other],creation_op='sub')
    return Tensor(self.data-other.data)
  def __mul__(self,other):
    if(self.autograd and other.autograd):
      return Tensor(self.data*other.data,autograd=True,creators=[self,other],creation_op='mul')
    return Tensor(self.data*other.data)
  def sum(self,dim):
    if(self.autograd):
      return Tensor(self.data.sum(dim),autograd=True,creators=[self],creation_op='sum_'+str(dim))
    return Tensor(self.data.sum(dim))
  def expand(self,dim,copies):
    trans_cmd = list(range(0,len(self.data.shape)))
    trans_cmd.insert(dim,len(self.data.shape))
    new_shape=list(self.data.shape) + [copies]
    new_data = self.data.repeat(copies).reshape(new_shape)
    new_data = new_data.transpose(trans_cmd)
    if (self.autograd):
      return Tensor(new_data,autograd=True,creators=[self],creation_op='expand_'+str(dim))
    return Tensor(new_data)
  def transpose(self):
    if (self.autograd):
      return Tensor(self.data.transpose(),autograd=True,creators=[self],creation_op='transpose')
    return Tensor(self.data.transpose())
  def mm(self,x):
    if (self.autograd):
      return Tensor(self.data.dot(x.data),autograd=True,creators=[self,x],creation_op='mm')
    return Tensor(self.data.dot(x.data))
#Добавить в backward новые функции

  def __repr__(self):
    return str(self.data.__repr__())
  def __str__(self):
    return str(self.data.__str__())

In [61]:
a=Tensor([1,2,3,4,5],autograd=True)
b=Tensor([2,2,2,2,2],autograd=True)
c=Tensor([5,4,3,2,1],autograd=True)
d=a+(-b)
e=(-b)+c
f=d+e
f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data)

[-2 -2 -2 -2 -2]


In [65]:
np.random.seed(0)
data=Tensor(np.array([[0,0],[0,1],[1,0],[1,1]]),autograd=True)
target=Tensor(np.array([[0],[1],[0],[1]]),autograd=True)
w=list()

w.append(Tensor(np.random.rand(2,3),autograd=True))
w.append(Tensor(np.random.rand(3,1),autograd=True))
for _ in range(20):
  pred=data.mm(w[0]).mm(w[1])
  loss = ((pred-target)*(pred-target)).sum(0)
  loss.backward(Tensor(np.ones_like(loss.data)))
  for ws in w:
    ws.data -= ws.grad.data*0.1
    ws.grad.data*=0.1
  print(loss)
               

[1.6973519]
[0.77451661]
[0.25897421]
[0.16823606]
[0.11731015]
[0.08220394]
[0.05619558]
[0.03747485]
[0.0243653]
[0.01546837]
[0.00960953]
[0.00585708]
[0.00351226]
[0.00207778]
[0.00121564]
[0.00070495]
[0.00040594]
[0.00023248]
[0.00013258]
[7.53627889e-05]


In [67]:
#стохастический градиентный спуск
class SGD(object):
  def __init__(self, parameters, alpha):
    self.parameters = parameters
    self.alpha = alpha
  def zero(self):
    for p in self.parameters:
      p.grad.data*=0
  def step(self,zero=True):
    for p in self.parameters:
      p.data -=  p.grad.data*self.alpha
      if zero:
        p.grad.data*=0

In [69]:
np.random.seed(0)
data=Tensor(np.array([[0,0],[0,1],[1,0],[1,1]]),autograd=True)
target=Tensor(np.array([[0],[1],[0],[1]]),autograd=True)
w=list()

w.append(Tensor(np.random.rand(2,3),autograd=True))
w.append(Tensor(np.random.rand(3,1),autograd=True))
optim=SGD(parameters=w, alpha=0.1)

for _ in range(20):
  pred=data.mm(w[0]).mm(w[1])
  loss = ((pred-target)*(pred-target)).sum(0)
  loss.backward(Tensor(np.ones_like(loss.data)))
  optim.step()
  print(loss)


[1.6973519]
[0.77451661]
[0.23280793]
[0.17652952]
[0.13235231]
[0.09761048]
[0.07068221]
[0.05021296]
[0.03499648]
[0.02394675]
[0.0161073]
[0.01066715]
[0.0069679]
[0.00449764]
[0.0028739]
[0.00182086]
[0.00114563]
[0.00071668]
[0.00044626]
[0.00027685]
